**<h1 style = 'text-align: center'>Week 02: Text Feature Extraction</h1>**

In [11]:
from sklearn.feature_extraction import DictVectorizer, FeatureHasher
import numpy as np
import scipy
import nltk
from nltk import pos_tag

## <b>1. Loading Features From Dicts </b>

In [25]:
# data in dict
data = [
    {'city': 'Dubai', 'temperature': 33.},
    {'city': 'London', 'temperature': 12.},
    {'city': 'San Francisco', 'temperature': 18.},
]

# init DictVectorizer
vect = DictVectorizer(sparse=True)     #sparse: đại diện cho ma trận thưa: chỉ lưu trữ những phần tử khác 0 và vị trí của nó

# apply on data
vectorized_data = vect.fit_transform(data)

# vect learns keys in data and stores them in inited DictVectorizer
print("Feature names:\n",vect.get_feature_names_out())         # get feature names
print()
print("Feature matrix:\n", vectorized_data.toarray())       # .toarray() giúp hiện ma trận đầy đủ


Feature names:
 ['city=Dubai' 'city=London' 'city=San Francisco' 'temperature']

Feature matrix:
 [[ 1.  0.  0. 33.]
 [ 0.  1.  0. 12.]
 [ 0.  0.  1. 18.]]


In [41]:
movie_entry = [{'category': ['thriller', 'drama'], 'year': 2003},
               {'category': ['animation', 'family'], 'year': 2011},
               {'year': 1974}]

vectorized_movie = vect.fit_transform(movie_entry)
print(vect.get_feature_names_out())
vectorized_movie.toarray()


test_data = {'category': ['thriller'],          # đặc trưng thriller có -> bật bit 1
               'unseen_feature': '3'}           # đặc trưng unseen_feature không có -> bỏ qua

# biến đổi dữ liệu test_data dựa trên đặc trưng đã học của movie_entry
vect.transform(test_data).toarray()


array([[0., 0., 0., 1., 0.]])

In [7]:
# extract Part of Speech
pos_window = [
    {
        'word-2': 'the',        # word là từ
        'pos-2': 'DT',          # pos (part of speech): từ loại của nó
        'word-1': 'cat',
        'pos-1': 'NN',
        'word+1': 'on',
        'pos+1': 'PP',
    },
    # in a real application one would extract many such dictionaries
]

vect = DictVectorizer()
pos_vectorized = vect.fit_transform(pos_window)

pos_vectorized.toarray()

vect.get_feature_names_out()


array(['pos+1=PP', 'pos-1=NN', 'pos-2=DT', 'word+1=on', 'word-1=cat',
       'word-2=the'], dtype=object)

## **2. Feature Hashing**

In [73]:
# tạo đối tượng feature cho từng token
def token_features(token, part_of_speech):
    if token.isdigit():     # nếu là token chữ số
        yield "numeric"
    else:                   # nếu là token chữ: đưa về lower -> thêm token & pos
        yield "token={}".format(token.lower())      
        yield "token,pos={},{}".format(token, part_of_speech)
    
    if token[0].isupper():      # kiểm tra chữ cái đầu in hoa
        yield "uppercase_initial"
    if token.isupper():
        yield "all_uppercase"
        
    yield "pos={}".format(part_of_speech)       # in pos của nó
    # cuối cùng nếu là số: thì token có "numeric" & "pos"
    # cuối cùng nếu là chữ: "token", "token,pos", "uppercase_initial" (hoặc "all_uppercase"), "pos"

corpus = "The quick brown fox jumps over the lazy dog."
tokens = nltk.word_tokenize(corpus)
pos_tagged = pos_tag(tokens)

raw_X = (token_features(tok, pos) for tok, pos in pos_tagged)
hasher = FeatureHasher(input_type='string', alternate_sign = True, )
X = hasher.transform(raw_X)     # X có kích thước lớn do FeatureHasher sử dụng bộ nhớ tối đa để lưu trữ dữ liệu -> 

print(type(X))

<class 'scipy.sparse._csr.csr_matrix'>


## <b>3. The Bag of Words</b>